# 

In [1]:
import os
from pathlib import Path

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models

from classification import get_cifar10_train_test_loader, train_epoch, validate_epoch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


/home/ubuntu/finetuning_cookbook_staging/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


## 事前準備
### ライブラリのインストール

In [2]:
%pip install pandas plotly torch torchvision

  Using cached torch-2.4.0-cp310-cp310-manylinux1_x86_64.whl (797.2 MB)
  Using cached torchvision-0.19.0-cp310-cp310-manylinux1_x86_64.whl (7.0 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 

In [2]:
def get_model(pretrained: bool = True, state_dict: dict | None = None):
    # 事前学習済みのResNetモデルをロード
    model = models.resnet50(pretrained=pretrained)
    # ResNetの最後の全結合層をクラス数に置き換え
    model.fc = nn.Linear(model.fc.in_features, 10)
    if state_dict is not None:
        model.load_state_dict(state_dict)

    # デバイスの選択、GPUが使用可能なら使う
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # 損失関数の設定
    criterion = nn.CrossEntropyLoss()
    # オプティマイザの設定
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    return model, criterion, optimizer

In [3]:
def run(
    train_samples: int = 1000,
    test_samples: int = 1000,
    pretrained: bool = True,
    num_epochs: int = 50,
):
    # 事前学習済みのResNetモデルをロード
    model, criterion, optimizer = get_model(pretrained=pretrained)

    # データをロードする
    train_loader, test_loader = get_cifar10_train_test_loader(
        train_samples=train_samples, test_samples=test_samples
    )

    result = []
    output_dir = Path(
        "output",
        "classification_cifar10",
        "pretrained" if pretrained else "un_pretrained",
        f"train_samples_{train_samples}",
    )

    os.makedirs(output_dir, exist_ok=True)
    for epoch in range(num_epochs):
        # 学習
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device)
        # 検証
        val_loss, predicted_output, true_label = validate_epoch(
            model, test_loader, criterion, device
        )
        # predicted_output と true_label から accuracy を計算する
        _, predicted_class = torch.max(predicted_output, dim=1)
        assert predicted_class.size() == true_label.size()
        correct = (predicted_class == true_label).sum().item()
        total = true_label.size(0)

        # 正解率の計算
        accuracy = correct / total
        result.append(
            {"train_loss": train_loss, "val_loss": val_loss, "accuracy": accuracy}
        )

        # 結果の表示
        print(f"Epoch {epoch+1}/{num_epochs}", result[-1])
        # モデルの保存
        # torch.save(model.state_dict(), output_dir / f"check_point_epoch_{epoch}.pt")

    df_result = pd.DataFrame(result)
    df_result.to_csv(output_dir / "training_curve.csv")

In [4]:
# 教師データの数を変えて実施
train_samples_list = [100, 500, 1000, 2000, 3000, 4000, 5000]

for num in train_samples_list:
    run(train_samples=num, test_samples=1000, pretrained=True)

/home/ubuntu/finetuning_cookbook_staging/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/finetuning_cookbook_staging/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


100%|██████████| 170498071/170498071 [00:24<00:00, 6935031.12it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


100%|██████████| 32/32 [00:41<00:00,  1.29s/it]


Epoch 1/50 {'train_loss': 2.3441266417503357, 'val_loss': 6.854854345321655, 'accuracy': 0.117}


100%|██████████| 32/32 [00:41<00:00,  1.31s/it]


Epoch 2/50 {'train_loss': 1.977068930864334, 'val_loss': 8.804275795817375, 'accuracy': 0.104}


 31%|███▏      | 10/32 [00:13<00:30,  1.37s/it]


KeyboardInterrupt: 

In [5]:
def load_result(pretrained: bool = True, train_samples: int = 1000):
    output_dir = Path(
        "output",
        "classification_cifar10",
        "pretrained" if pretrained else "un_pretrained",
        f"train_samples_{train_samples}",
    )
    df_result = pd.read_csv(output_dir / "training_curve.csv", index_col=0)
    return df_result


def plot_result(pretrained: bool = True, train_samples: int = 1000):
    df = load_result(pretrained=pretrained, train_samples=train_samples)
    title = f"{pretrained=} {train_samples=}"
    fig_loss = px.line(df, y=["train_loss", "val_loss"])
    fig_loss.update_layout(
        title=f"Loss {title}",
        xaxis_title="Epoch",
        yaxis_title="Loss",
        yaxis_range=[0, 5],
    )
    fig_loss.write_image(
        f"output/classification_cifar10/figs/loss_{title.replace(' ', '_')}.png"
    )
    fig_acc = px.line(df, y=["accuracy"])
    fig_acc.update_layout(
        title=f"Accuracy {title}",
        xaxis_title="Epoch",
        yaxis_title="Test Accuracy",
        yaxis_range=[0, 1],
    )
    fig_acc.write_image(
        f"output/classification_cifar10/figs/acc_{title.replace(' ', '_')}.png"
    )

    return fig_loss, fig_acc


def plot_mix_result(train_samples: int = 1000):
    df_fine = load_result(pretrained=True, train_samples=train_samples)
    df_no_fine = load_result(pretrained=False, train_samples=train_samples)

    fig = go.Figure()
    fig.add_scatter(x=df_fine.index, y=df_fine["accuracy"], name="fine tuning accuracy")
    fig.add_scatter(
        x=df_fine.index, y=df_no_fine["accuracy"], name="no fine tuning accuracy"
    )
    title = f"{train_samples=}"
    fig.update_layout(
        title=f"Accuracy {title}",
        xaxis_title="Epoch",
        yaxis_title="Accuracy",
        yaxis_range=[0, 1],
    )
    fig.write_image(f"output/classification_cifar10/figs/acc_fine_{title}.png")

    return fig

In [6]:
for samples in [100, 500, 1000, 2000, 3000, 4000, 5000]:
    plot_result(pretrained=True, train_samples=samples)
    plot_result(pretrained=False, train_samples=samples)
    plot_mix_result(train_samples=samples)

FileNotFoundError: [Errno 2] No such file or directory: 'output/classification_cifar10/pretrained/train_samples_100/training_curve.csv'

In [7]:
plot_result(pretrained=True, train_samples=500)
plot_result(pretrained=False, train_samples=500)

FileNotFoundError: [Errno 2] No such file or directory: 'output/classification_cifar10/pretrained/train_samples_500/training_curve.csv'

In [8]:
# データ量変更実験の可視化
pretrain_max_list = []
no_pretrain_max_list = []
for samples in [100, 500, 1000, 2000, 3000, 4000, 5000]:
    pretrain_max_list.append(
        load_result(pretrained=True, train_samples=samples)["accuracy"].max()
    )
    no_pretrain_max_list.append(
        load_result(pretrained=False, train_samples=samples)["accuracy"].max()
    )
df_max = pd.DataFrame(
    {
        "pretrain_max": pretrain_max_list,
        "no_pretrain_max": no_pretrain_max_list,
        "train_samples": [100, 500, 1000, 2000, 3000, 4000, 5000],
    },
    index=range(7),
)
fig = px.scatter(df_max, y=["pretrain_max", "no_pretrain_max"], x="train_samples")
fig.update_layout(
    title="学習サンプルサイズと精度",
    xaxis_title="Train Samples",
    yaxis_title="Accuracy",
    yaxis_range=[0, 1],
)
fig.write_image("output/classification_cifar10/figs/acc_max.png")
fig.show()

FileNotFoundError: [Errno 2] No such file or directory: 'output/classification_cifar10/pretrained/train_samples_100/training_curve.csv'

In [9]:
def max_acc(df) -> float:
    return df["accuracy"].max()

def max_acc_list(train_samples=samples):
    return load_result(pretrained=True, train_samples=train_samples)


(map(lambda x: max_acc(load_result(pretrained=True, train_samples=x)), [100, 500]))

In [10]:
load_result(train_samples=100)


FileNotFoundError: [Errno 2] No such file or directory: 'output/classification_cifar10/pretrained/train_samples_100/training_curve.csv'